In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

In [2]:
!unzip -q library_dataset_all_2019-04-11_17-20-18.zip

In [3]:
def get_steering(path):
    return (float(int(path[9:12])) - 50.0) / 50.0

def get_gaussian(mean, stdev, nbins):
    x = np.linspace(-1.0, 1.0, num=nbins)
    y = np.exp(-(x - mean)**2 / (stdev**2))
    return y

def get_bin(value, nbins):
    return np.digitize(value, np.linspace(-1.0, 1.0, num=nbins))

class SteeringDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, stdev=0.3, nbins=21):
        self.nbins = nbins
        self.stdev = stdev
        self.directory = directory
        self.image_paths = glob.glob(os.path.join(directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.1, 0.1, 0.1, 0.1)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = PIL.Image.open(image_path)
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        steering = float(get_steering(os.path.basename(image_path)))
        #target = get_gaussian(steering, self.stdev, self.nbins)
        #target = get_bin(steering, self.nbins)
        
        return image, torch.tensor([steering]).float()
    
    
class MergedSteeringDataset(torch.utils.data.Dataset):
    
    def __init__(self, all_dir, task_dir, flipped=False, random_all_flips=False):
        self.flipped = flipped
        self.all_dir = all_dir
        self.task_dir = task_dir
        self.all_image_paths = glob.glob(os.path.join(self.all_dir, '*.jpg'))
        self.task_image_paths = glob.glob(os.path.join(self.task_dir, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
        self.random_all_flips = random_all_flips
    
    def __len__(self):
        return len(self.all_image_paths) + len(self.task_image_paths)
    
    def __getitem__(self, idx):
        flip_all = False
        if idx < len(self.all_image_paths):
            image_path = self.all_image_paths[idx]
            
            if float(np.random.rand(1)) > 0.5:
                flip_all = True and self.random_all_flips
        else:
            image_path = self.task_image_paths[idx - len(self.all_image_paths)]
        
        image = PIL.Image.open(image_path)
        steering = float(get_steering(os.path.basename(image_path)))
        
        if self.flipped:
            image = transforms.functional.hflip(image)
            steering = -steering
        
        if flip_all:
            image = transforms.functional.hflip(image)
            steering = -steering
            
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        #target = get_gaussian(steering, self.stdev, self.nbins)
        #target = get_bin(steering, self.nbins)
        
        return image, torch.tensor([steering]).float()

In [4]:
dataset = MergedSteeringDataset('dataset_all', 'dataset_forward', flipped=False)

test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

In [5]:
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 1)
device = torch.device('cuda')
model = model.to(device)

# train regression:

In [6]:

#optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
optimizer = optim.Adam(model.parameters())

In [7]:
NUM_EPOCHS = 50
BEST_MODEL_PATH = 'best_steering_model_library.pth'
best_loss = 1e9


for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += loss
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += loss
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

2.577181, 1.929390
3.340269, 3.460877
0.544457, 69.827301
0.297309, 29.402124
0.153697, 1.542875
0.148374, 1.863160
0.101047, 8.614643
0.079736, 1.569676
0.084514, 0.676401
0.070824, 0.387209
0.053442, 0.666871
0.049908, 0.534838
0.048813, 0.076598
0.053839, 0.115044
0.060421, 0.109403
0.053893, 0.038203
0.033368, 0.057634
0.031288, 0.044561
0.024159, 0.070578
0.025651, 0.030458
0.019782, 0.046745
0.023424, 0.036830
0.015258, 0.044347
0.018040, 0.034800
0.028941, 0.040266
0.012496, 0.049888
0.011510, 0.037328
0.016229, 0.037234
0.018740, 0.025118
0.014507, 0.045985
0.008088, 0.027261
0.011165, 0.032607
0.014194, 0.029452
0.017361, 0.029879
0.016522, 0.045812
0.013572, 0.046147
0.009901, 0.037356
0.020727, 0.023578
0.012250, 0.051145
0.011577, 0.026466
0.017888, 0.036059
0.009573, 0.042519
0.010715, 0.033554
0.008241, 0.028503
0.022338, 0.052088
0.012313, 0.038445
0.016847, 0.036897
0.008228, 0.040471
0.011586, 0.027539
0.012387, 0.032859


In [10]:
torch.save(model.state_dict(), BEST_MODEL_PATH)

In [17]:
torch.save(model.state_dict(), 'best_steering_model_forward_merged_end.pth')